# Proyecto 2
## RSNA 2022 Cervical Spine Fracture Detection
**Universidad del Valle de Guatemala**\
**Facultad de Ingeniería**\
**Departamento de Ciencias de la Computación**\
**Data Science**

# Generador de Metadatos
---

## Integrantes
- Gustavo Gonzalez
- Pablo Orellana
- Diego Leiva
- Maria Ramirez

---


## Librerias

In [1]:
# Operaciones de archivos
import os
from glob import glob

# Operaciones de datos
import pandas as pd
import numpy as np

# Operaciones de imágenes
import pydicom

# Visualización
from tqdm import tqdm

# Warnings
import warnings
warnings.filterwarnings('ignore')

# Metadatos DICOM

En la fase de análisis exploratorio se descubrió que las imágenes DICOM contienen etiquetas relevantes asociadas a metadatos específicos de la tomografía. Por lo tanto, se extraerá la información relevante para utilizar estos metadatos en una posterior detección de vértebras, aprovechando la información proporcionada.

## Lectura de Metadatos

In [2]:
meta_train = pd.read_csv("data/meta_train.csv")
print("Metadata cargado exitosamente")

Metadata cargado exitosamente


In [3]:
meta_train.head()

,Rows,Columns,SOPInstanceUID,ContentDate,SliceThickness,InstanceNumber,ImagePositionPatient,ImageOrientationPatient
0,512,512,1.2.826.0.1.3680043.6200.1.1,20220727,1.0,1,"[-118.530, -103.5302, 334.50]","[1.00000, 0.00000, 0.00000, 0.00000, 1.00000, ..."
1,512,512,1.2.826.0.1.3680043.6200.1.10,20220727,1.0,10,"[-118.530, -103.5302, 327.30]","[1.00000, 0.00000, 0.00000, 0.00000, 1.00000, ..."
2,512,512,1.2.826.0.1.3680043.6200.1.100,20220727,1.0,100,"[-118.530, -103.5302, 255.30]","[1.00000, 0.00000, 0.00000, 0.00000, 1.00000, ..."
3,512,512,1.2.826.0.1.3680043.6200.1.101,20220727,1.0,101,"[-118.530, -103.5302, 254.50]","[1.00000, 0.00000, 0.00000, 0.00000, 1.00000, ..."
4,512,512,1.2.826.0.1.3680043.6200.1.102,20220727,1.0,102,"[-118.530, -103.5302, 253.70]","[1.00000, 0.00000, 0.00000, 0.00000, 1.00000, ..."


## Procesamiento de Metadatos

### Columnas de Identificación y Tamaño de Imagen

In [4]:
# Generar columna del ID del estudio a partir del ID de la imagen
meta_train["StudyInstanceUID"] = meta_train["SOPInstanceUID"].apply(lambda x: ".".join(x.split(".")[:-2]))

# Generar columna del tamaño de la imagen a partir de las filas y columnas
meta_train["ImageSize"] = meta_train["Rows"].astype(str) + " x " + meta_train["Columns"].astype(str)

### Extracción de Coordenadas de Posición del Paciente

In [5]:
# Extraer las coordenadas de la posición de la imagen en los ejes x, y, z
meta_train['ImagePositionPatient_x'] = meta_train['ImagePositionPatient'].apply(lambda x: float(x.replace(',','').replace(']','').replace('[','').split()[0]))
meta_train['ImagePositionPatient_y'] = meta_train['ImagePositionPatient'].apply(lambda x: float(x.replace(',','').replace(']','').replace('[','').split()[1]))
meta_train['ImagePositionPatient_z'] = meta_train['ImagePositionPatient'].apply(lambda x: float(x.replace(',','').replace(']','').replace('[','').split()[2]))

In [6]:
# Eliminar columnas innecesarias reemplazadas por las nuevas
meta_train_clean = meta_train.drop(['SOPInstanceUID','ImagePositionPatient','ImageOrientationPatient','ImageSize','ContentDate'], axis=1)
# Renombrar columnas para mayor claridad
meta_train_clean.rename(columns={"Rows": "ImageHeight", "Columns": "ImageWidth","InstanceNumber": "Slice"}, inplace=True)
# Reordenar las columnas
meta_train_clean = meta_train_clean[['StudyInstanceUID','Slice','ImageHeight','ImageWidth','SliceThickness','ImagePositionPatient_x','ImagePositionPatient_y','ImagePositionPatient_z']]
# Ordenar los datos por ID de estudio y número de corte
meta_train_clean.sort_values(by=['StudyInstanceUID','Slice'], inplace=True)
# Resetear el índice
meta_train_clean.reset_index(drop=True, inplace=True)

In [7]:
meta_train_clean.head()

,StudyInstanceUID,Slice,ImageHeight,ImageWidth,SliceThickness,ImagePositionPatient_x,ImagePositionPatient_y,ImagePositionPatient_z
0,1.2.826.0.1.3680043.10001,1,512,512,0.625,-52.308,-27.712,7.282
1,1.2.826.0.1.3680043.10001,2,512,512,0.625,-52.308,-27.712,6.657
2,1.2.826.0.1.3680043.10001,3,512,512,0.625,-52.308,-27.712,6.032
3,1.2.826.0.1.3680043.10001,4,512,512,0.625,-52.308,-27.712,5.407
4,1.2.826.0.1.3680043.10001,5,512,512,0.625,-52.308,-27.712,4.782


## Exportacion de Metadatos Limpios

In [8]:
# Guardar el archivo limpio en formato CSV
meta_train_clean.to_csv("metadata/dicom_metadata.csv", index=False)